# Final Project Submission

* Student name: Cody Freese
* Student pace: part time
* Instructor name: James Irving
* Blog post URL:

## TABLE OF CONTENTS 

*Click to jump to matching Markdown Header.*<br><br>

<font size=3rem>
    
- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
</font>
___

# INTRODUCTION

Your target audience is non-technical people interested in using your findings to maximize their profit when selling their home.

-Does the Grade scale has significant impact on the sell price of the house? (The factors that influence the grade scale, what can the homeowner do on their end to improve said score)

-Does the number of bedrooms have a significant impant on the sell price of the house? (Would adding another room/wing on make it worth the investment)

-Does the number of bathrooms have a significant impant on the sell price of the house? (Would adding another bathroom or half bath on make it worth the investment)



# OBTAIN

## Import tools

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
from scipy.stats import zscore
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## King County dataset

In [2]:
df = pd.read_csv('kc_house_data.csv')
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


# SCRUB

### Drop
- Factors that homeowners themselves cannot renovate or control

In [ ]:
df.drop(['date','view','zipcode', 'lat','sqft_basement', 'long','sqft_lot', 'waterfront', 'sqft_living15', 'sqft_lot15' ],axis=1,inplace=True)
df

In [ ]:
df.info()

### Info
- Dates are objects
- Price includes cents
- Bedrooms are whole values
- Bathrooms are floats, i.e. 2 1/2 bath
- Sqft_living is whole value
- floors is floats
- condition is whole value, related to scale system
- grade is whole value, related to scale system
- sqft_above is whole value
- yr_built is integer = year
- yr_renovated is float, convert to datetime

### Check for NaN

In [ ]:
df.isna().sum()

### Drop Null Values

In [ ]:
#Drop NaN
df_renovated = df.copy()
df_renovated = df_renovated.dropna()
df_renovated

### Convert Year Renovated to Integer

In [ ]:
#Ensuring no strings in yr_renovated
df_renovated['yr_renovated'] = df_renovated['yr_renovated'].astype('float')
#Convert to integer for whole number year
df_renovated['yr_renovated'] = df_renovated['yr_renovated'].astype('int')

In [ ]:
df_renovated.info()

### Datetime

In [ ]:
#df_renovated['date'] = pd.to_datetime(df_renovated['date'])
#df_renovated['date']

In [ ]:
for col in df_renovated.columns:
    try:
        print(col, df_renovated[col].value_counts()[:5])
    except:
        print(col, df_renovated[col].value_counts())
    print('\n')

### Check for duplicates

In [ ]:
duplicates = df_renovated[df_renovated.duplicated()]
print(len(duplicates))
duplicates.head()

### Value Counts

In [ ]:
# Value counts for variables
for col in df_renovated.columns:
    print(col, '\n', df[col].value_counts(normalize=True).head(), '\n\n')

### Function on Variables

In [ ]:
def check_column(df_renovated, col_name, n_unique=10):
    print('Datatype')
    print('\t',df_renovated[col_name].dtypes)
    
    num_nulls = df_renovated[col_name].isna().sum()
    print(f'Null Values Present = {num_nulls}')
    
    display(df_renovated[col_name].describe().round(3))
    
    print('\nValue Counts:')
    display(df_renovated[col_name].value_counts(n_unique))
    
check_column(df_renovated,'price')

### Summary

In [ ]:
df_renovated.describe().round(3)

- Price
    - Average - $540,660.02
    
    - Median - $450,000.00
    
    - STD - 369300.621
    
- Bedrooms
    - Average - 3.378
    
    - Median - 3.000
    
    - STD - 0.933
    
- Bathrooms
    - Average - 2.121
    
    - Median - 2.250
    
    - STD - 0.768
    
- SqFt_Living
    - Average - 2084.291
    
    - Median - 1920.000
    
    - STD - 917.594
    
- Floors
    - Average - 1.496
    
    - Median - 1.500
    
    - STD - 0.538
- Condition
    - Average - 3.411
    
    - Median - 3.000
    
    - STD - 0.652
- Grade
    - Average - 7.662
    
    - Median - 7.000
    
    - STD - 1.171
    
    - **Range** 3-13

- Sqft_above
    - Average - 1791.563
    
    - Median - 1570.000
    
    - STD - 826.690

- Yr_Built
    - Min - 1900
    
    - Average - 1971
    
    - Median - 1975
    
    - Max - 2015
    
    - STD - 29.353


# EXPLORE

## Histogram

In [ ]:
df_renovated.hist(figsize=(10,10))
plt.tight_layout()

## Scatter Matrix

In [ ]:
fig = pd.plotting.scatter_matrix(df_renovated,figsize=(16,16));
print(type(fig))

## Heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

corr = df_renovated.corr().abs().round(3)

mask = np.triu(np.ones_like(corr, dtype=np.bool))

sns.heatmap(corr, annot=True, mask=mask, cmap='Oranges', ax=ax)
plt.setp(ax.get_xticklabels(), 
         rotation=45, 
         ha="right",
         rotation_mode="anchor")
ax.set_title('Correlations')

## Scatter Target Variables

In [ ]:
cols_to_plot = ["bedrooms", "bathrooms","grade","floors", "condition"]

colors = ['red', 'orange', 'magenta', 'green', 'blue']
fig, axes = plt.subplots(ncols=5, figsize=(16,3))

for i, col in enumerate(cols_to_plot):
    axes[i].scatter(x=df_renovated[col],y=df_renovated['price'],c=colors[i],marker='.')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('price')
    
plt.tight_layout()

## Grade/Price Plot

In [ ]:
sns.regplot(x= 'grade', y= 'price', data=df_renovated)

## Normalization?

In [ ]:
df_renovated.info()

In [ ]:
#Normalizing? Not Sure if Needed

#df_whatisthis = df_renovated.copy()
#df_whatisthis.columns = [col.replace(' ', '_') for col in df_whatisthis.columns]
#for col in x_cols:
#    df_whatisthis[col] = (df_whatisthis[col] - df_whatisthis[col].mean())/df_whatisthis[col].std()
#df_whatisthis.head()

# MODEL

## Initial Model

In [ ]:
outcome = 'price'
x_cols = ['bedrooms','bathrooms','floors','condition','grade','yr_built','yr_renovated','sqft_living', 'sqft_above']

In [ ]:
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=df_renovated).fit()
model.summary()

## Regression Results
- Skew:	3.513
- Kurtosis:	41.971
- R-squared:	0.617

In [ ]:
model.params.sort_values()

## Initial Trends

- Bedrooms & square footage of house apart from basement has negative impact on price

- $539,809.47 intercept

- sqft_living (square footage of the home) has largest impact

- Grade

- Bathrooms

- Floors

## Refinement

### Normality

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

### Outliers Present
- Check Prices

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(df_renovated['price'].value_counts().sort_index())

### Filter price
- Try to filter over $1,000,000

In [ ]:
for i in range(90,99):
    q = i / 100
    print('{} percentile: {}'.format(q, df_renovated['price'].quantile(q=q)))

### Interquartile Range

In [ ]:
q1 = df_renovated.quantile(0.05)
q3 = df_renovated.quantile(0.95)
IQR = q3 - q1
print(IQR)

In [ ]:
print(q1 - 1.5 * IQR)

In [ ]:
print(q3 + 1.5 * IQR)

In [ ]:
print(df_renovated < (q1 - 1.5 * IQR)) | (df_renovated > (q3 + 1.5 * IQR))

In [ ]:
z = np.abs(stats.zscore(df_renovated))
df_z = df_renovated[(z < 3).all(axis=1)]
df_IQR = df_z[~((df_z < (q1 - 1.5 * IQR)) |(df_z > (q3 + 1.5 * IQR))).any(axis=1)]
subset = df_IQR[(df_IQR['price'] < 1000000)] 
print('Percent removed:', (len(df_renovated) - len(subset))/len(df_renovated))
outcome = 'price'
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=subset).fit()
model.summary()

In [ ]:
name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
test = sms.jarque_bera(model.resid)
list(zip(name, test))

### Homoscedasticity

In [ ]:
plt.scatter(model.predict(subset[x_cols]), model.resid)
plt.plot(model.predict(subset[x_cols]), [0 for i in range(len(subset))])

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)

## Train Split Test

In [ ]:
y = df_renovated[['price']]
X = df_renovated.drop(['price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
#print(X_train)

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

In [ ]:
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test = np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

In [ ]:
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

In [ ]:
linreg.score(X_test, y_test)

### Z - Score

In [ ]:
z = np.abs(stats.zscore(df_renovated["price"]))
print(np.where(z > 3))

In [ ]:
df_renovated

# Interpret

In [ ]:
model.summary()

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_renovated = df_renovated.sort_values('coef', ascending=False)
df_renovated.head(15)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='grade', y='price', data=df_renovated)
ax.set(title='Grade relationship on Price', 
       xlabel='Grade', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='bathrooms', y='price', data=df_renovated)
ax.set(title='Bathrooms & Price', 
       xlabel='Bathrooms', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='bedrooms', y='price', data=df_renovated)
ax.set(title='Bedrooms & Price', 
       xlabel='Bedrooms', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='floors', y='price', data=df_renovated)
ax.set(title='Floors & Price', 
       xlabel='Floors', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='condition', y='price', data=df_renovated)
ax.set(title='Condition & Price', 
       xlabel='Condition', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.regplot(x='sqft_living', y='price', data=df_renovated)
ax.set(title='Square Feet Living Space & Price', 
       xlabel='SqFt.', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.regplot(x='sqft_above', y='price', data=df_renovated)
ax.set(title='Square Feet Above & Price', 
       xlabel='Sqft. Above', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='yr_built', y='price', data=df_renovated)
ax.set(title='Year Built & Price', 
       xlabel='Year Built', ylabel='Price')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.boxplot(x='yr_renovated', y='price', data=df_renovated)
ax.set(title='Year Renovated & Price', 
       xlabel='Year', ylabel='Price')

fig.tight_layout()

# CONCLUSIONS & RECOMMENDATIONS

> Summarize your conclusions and bullet-point your list of recommendations, which are based on your modeling results.

- Higher grade rating increases the value of a home
- The number of bathrooms increases the value of a home
- Having more available SqFt. Living space increases the value of a home.

- Maximize available square footage by opening up rooms more or freeing up unused space.
- If specialized rooms outside living area are desired, bathrooms appear to have the most value toward future sell value
- Following Grade guidelines to increase score will inscrease the value of your home